In [11]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import json
g_key = ""
gmaps.configure(api_key=g_key)

### Store Part I results

In [12]:
# read the cities.csv to create dataframe, previous cells are not needed to be executed as long as cities.csv was created
cities = pd.read_csv("output_data/cities.csv", encoding="utf-8")
cities.head()

,City,Country,Lat,Lng,Date,Cloudiness,Humidity,Max Temp,Wind Speed
0,dalbandin,PK,28.894699,64.410088,1578960000,0,67,36.19,9.80
1,sao joao da barra,BR,-21.640280,-41.051109,1578960000,100,92,76.21,9.42
2,port alfred,ZA,-33.590569,26.891041,1578960000,80,90,72.36,17.74
3,hilo,US,19.729719,-155.089996,1578960000,100,80,72.61,8.61
4,pouebo,NC,-20.400000,164.566666,1578960000,90,54,84.70,9.51


### Humidity Heatmap

In [13]:
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]

In [14]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [15]:
narrowed_city_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,City,Country,Lat,Lng,Date,Cloudiness,Humidity,Max Temp,Wind Speed
0,yenagoa,NG,4.924720,6.264170,1578960000,0,87,78.44,2.80
1,rikitea,PF,-23.120279,-134.969177,1578960000,0,80,78.04,10.09
2,caravelas,BR,-17.712549,-39.248138,1578960000,0,88,79.47,13.58
3,takoradi,GH,4.884470,-1.755360,1578960000,0,87,78.28,3.53
4,port blair,IN,11.666670,92.750000,1578960000,0,75,78.69,10.16


### Hotel Map

In [16]:
hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,City,Country,Lat,Lng,Date,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name
0,yenagoa,NG,4.924720,6.264170,1578960000,0,87,78.44,2.80,Yenagoa
1,rikitea,PF,-23.120279,-134.969177,1578960000,0,80,78.04,10.09,Rikitea
2,caravelas,BR,-17.712549,-39.248138,1578960000,0,88,79.47,13.58,Caravelas
3,takoradi,GH,4.884470,-1.755360,1578960000,0,87,78.28,3.53,Takoradi
4,port blair,IN,11.666670,92.750000,1578960000,0,75,78.69,10.16,Port Blair


In [19]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

In [18]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))